In [1]:
import os
import numpy as np
import pandas as pd
import hickle as hkl

from collections import defaultdict

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, explained_variance_score, mean_absolute_error, mean_squared_error

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, Dropout, LSTM, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

from IPython.core.display import display, HTML



# Hide GPU from visible devices

#'''
tf.config.set_visible_devices([], 'GPU')
print(f'CUDA GPU AVAILABLE: {tf.test.is_gpu_available(cuda_only=True)}')
'''
THREADS = 8
os.environ['OMP_NUM_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTEROP_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.set_soft_device_placement(True)
'''

#'''
display(HTML("<style>.container { width:100% !important; }</style>"))
#'''

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CUDA GPU AVAILABLE: True


In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true)

In [3]:
N_SPLITS = 5
N_STEPS = 20
N_DAYS = 60
DATA_DIR = f'data/processed_data/cv_data__{N_SPLITS}_class_id_folds__{N_STEPS}_steps__{N_DAYS}_days'

In [4]:
results = []

for i in range(N_SPLITS):

    # Load one partition of folds
    data = hkl.load(f'{DATA_DIR}/fold_{i}.hkl')
    action_training_input = data['action_training_input']
    recurrent_training_input = data['recurrent_training_input']
    prior_training_input = data['prior_training_input']
    completion_training_target = data['completion_training_target']
    problems_training_target = data['problems_training_target']
    training_target_sequence = data['training_target_sequence']
    action_testing_input = data['action_testing_input']
    recurrent_testing_input = data['recurrent_testing_input']
    prior_testing_input = data['prior_testing_input']
    completion_testing_target = data['completion_testing_target']
    problems_testing_target = data['problems_testing_target']
    testing_target_sequence = data['testing_target_sequence']
    
    # Clear session so models don't pile up
    keras.backend.clear_session()

    # Create model
    
    # Action Model
    action_input_layer = Input(shape=action_training_input[0].shape, name='action')
    action_model = Dropout(rate=0.5)(action_input_layer)
    action_model = LSTM(units=64, return_sequences=False, activation='relu', dropout=0.5, recurrent_dropout=0.5)(action_model)
    
    # Recurrent Input
    recurrent_input_layer = Input(shape=recurrent_training_input[0].shape, name='recurrent')
    recurrent_model = Masking(mask_value=0.0)(recurrent_input_layer)
    recurrent_model = Dropout(rate=0.5)(recurrent_model)
    recurrent_model = LSTM(units=64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)(recurrent_model)
    
    # Prior Input
    prior_input_layer = Input(shape=prior_training_input[0].shape, name='prior')
    prior_model = Dropout(rate=0.5)(prior_input_layer)
    prior_model = Dense(units=64, activation='sigmoid')(prior_model)
    
    # Combined Model
    model = Concatenate()([action_model, recurrent_model, prior_model])
    model = Dropout(rate=0.5)(model)
    
    # Combined Output
    completion_output_layer = Dense(units=1, activation='sigmoid', name='completion')(model)
    problems_output_layer = Dense(units=1, activation='linear', name='problems')(model)
    
    combined_model = Model([action_input_layer, recurrent_input_layer, prior_input_layer], [completion_output_layer, problems_output_layer])
    combined_model.compile(optimizer='adam', loss={'completion': 'binary_crossentropy', 'problems': 'mse'})

    # Train model
    es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]
    weights = {'completion': np.ones_like(completion_training_target) * 16, 'problems': completion_training_target}
    combined_model.fit(x={'action': action_training_input, 'recurrent': recurrent_training_input, 'prior': prior_training_input},
                       y={'completion': completion_training_target, 'problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    # Store model predictions
    completion_testing_output, problems_testing_output = combined_model.predict({'action': action_testing_input, 
                                                                                 'recurrent': recurrent_testing_input, 
                                                                                 'prior': prior_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    

pd.concat(results).to_csv('cross_validation_results.csv', index=False)

Epoch 1/1000
3949/3949 [==============================] - 271s 68ms/step - loss: 15.1359 - completion_loss: 7.9449 - problems_loss: 7.1911 - val_loss: 14.2011 - val_completion_loss: 8.5375 - val_problems_loss: 5.6637
Epoch 2/1000
3949/3949 [==============================] - 264s 67ms/step - loss: 14.6680 - completion_loss: 7.7603 - problems_loss: 6.9077 - val_loss: 14.1311 - val_completion_loss: 8.4941 - val_problems_loss: 5.6370
Epoch 3/1000
3949/3949 [==============================] - 262s 66ms/step - loss: 14.5776 - completion_loss: 7.7179 - problems_loss: 6.8597 - val_loss: 14.1928 - val_completion_loss: 8.5482 - val_problems_loss: 5.6446
Epoch 4/1000
3949/3949 [==============================] - 260s 66ms/step - loss: 14.5079 - completion_loss: 7.6785 - problems_loss: 6.8294 - val_loss: 14.0898 - val_completion_loss: 8.4729 - val_problems_loss: 5.6169
Epoch 5/1000
3949/3949 [==============================] - 260s 66ms/step - loss: 14.4813 - completion_loss: 7.6683 - problems_loss: 

Epoch 16/1000
3949/3949 [==============================] - 254s 64ms/step - loss: 13.8816 - completion_loss: 7.6010 - problems_loss: 6.2805 - val_loss: 13.9159 - val_completion_loss: 8.4359 - val_problems_loss: 5.4800
Epoch 17/1000
3949/3949 [==============================] - 254s 64ms/step - loss: 13.8489 - completion_loss: 7.5792 - problems_loss: 6.2697 - val_loss: 13.8905 - val_completion_loss: 8.4040 - val_problems_loss: 5.4865
Epoch 18/1000
3949/3949 [==============================] - 254s 64ms/step - loss: 13.8543 - completion_loss: 7.5803 - problems_loss: 6.2740 - val_loss: 13.8518 - val_completion_loss: 8.3791 - val_problems_loss: 5.4727
Epoch 1/1000
3949/3949 [==============================] - 266s 67ms/step - loss: 14.4822 - completion_loss: 8.0058 - problems_loss: 6.4764 - val_loss: 14.0670 - val_completion_loss: 8.2882 - val_problems_loss: 5.7788
Epoch 2/1000
3949/3949 [==============================] - 261s 66ms/step - loss: 14.0582 - completion_loss: 7.8241 - problems_los

3949/3949 [==============================] - 262s 66ms/step - loss: 15.3219 - completion_loss: 7.9405 - problems_loss: 7.3814 - val_loss: 13.9808 - val_completion_loss: 8.4187 - val_problems_loss: 5.5621
Epoch 3/1000
3949/3949 [==============================] - 262s 66ms/step - loss: 14.7862 - completion_loss: 7.8080 - problems_loss: 6.9782 - val_loss: 13.9118 - val_completion_loss: 8.3975 - val_problems_loss: 5.5143
Epoch 4/1000
3949/3949 [==============================] - 262s 66ms/step - loss: 14.6243 - completion_loss: 7.7603 - problems_loss: 6.8640 - val_loss: 13.9202 - val_completion_loss: 8.4026 - val_problems_loss: 5.5175
Epoch 5/1000
3949/3949 [==============================] - 262s 66ms/step - loss: 14.5194 - completion_loss: 7.7116 - problems_loss: 6.8078 - val_loss: 13.9424 - val_completion_loss: 8.3741 - val_problems_loss: 5.5684
Epoch 6/1000
3949/3949 [==============================] - 262s 66ms/step - loss: 14.4677 - completion_loss: 7.6895 - problems_loss: 6.7781 - val_

In [5]:
# Load the data

exp_norm_map = pd.read_csv('data/experiment_information/exp_norm_map.csv')
results = pd.read_csv('cross_validation_results.csv')

In [6]:
# Evaluate the results

norm_exp_dict = defaultdict(lambda: 'None')
for n, e in zip(exp_norm_map['normal_id'], exp_norm_map['experiment_id']):
    norm_exp_dict[n] = e
results['experiment_sequence'] = results['target_sequence'].map(norm_exp_dict)

metrics = []
0
# Calculate the metrics for each sequence
for sequence, df in results.groupby('target_sequence'):
    
    completion_target = df['completion_target']
    
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([sequence, 
                    norm_exp_dict[sequence], 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for each fold
for fold, df in results.groupby('fold'):
    
    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([f'fold_{fold}', 
                    'None', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for everything combined
completion_target = results['completion_target']
completion_prediction = results['completion_prediction']
completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_mse = mean_squared_error(completion_target, completion_prediction)

problems_target = results[results['completion_target'] == 1]['problems_target']
problems_prediction = results[results['completion_target'] == 1]['problems_prediction']
problems_mae = mean_absolute_error(problems_target, problems_prediction)
problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_mse = mean_squared_error(problems_target, problems_prediction)

metrics.append(['all_target_data', 
                'None', 
                len(results), 
                completion_auc, 
                completion_acc, 
                completion_r2, 
                completion_ev, 
                completion_mse, 
                problems_mae, 
                problems_mape, 
                problems_r2, 
                problems_ev, 
                problems_mse])

metrics = pd.DataFrame(metrics, 
                       columns=['sequence_id', 
                                'experiment_id', 
                                'sample_size', 
                                'completion_auc', 
                                'completion_acc', 
                                'completion_r2', 
                                'completion_ev', 
                                'completion_mse', 
                                'problems_mae', 
                                'problems_mape', 
                                'problems_r2', 
                                'problems_ev', 
                                'problems_mse'])

metrics.to_csv('cross_validation_metrics.csv', index=False)

In [7]:
metrics.iloc[-6:]

,sequence_id,experiment_id,sample_size,completion_auc,completion_acc,completion_r2,completion_ev,completion_mse,problems_mae,problems_mape,problems_r2,problems_ev,problems_mse
288,fold_0,None,42114,0.765085,0.774968,0.173501,0.173502,0.154738,1.911902,0.221846,0.039910,0.040975,7.267810
289,fold_1,None,42114,0.767978,0.773187,0.179868,0.180521,0.156128,1.965523,0.222409,0.038601,0.038704,9.174119
290,fold_2,None,42114,0.766207,0.773662,0.176731,0.177197,0.156141,1.942109,0.185738,0.033547,0.035378,9.899226
291,fold_3,None,42113,0.760770,0.764989,0.167001,0.167049,0.159999,1.876787,0.177404,0.038517,0.039994,7.581959
292,fold_4,None,42113,0.762857,0.771306,0.170839,0.170872,0.157724,1.911022,0.183111,0.041704,0.043150,7.798485
293,all_target_data,None,210568,0.764437,0.771622,0.173619,0.173669,0.156946,1.921502,0.198156,0.038347,0.038593,8.343768
